<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ резюме из HeadHunter
   

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [ ]:
import pandas as pd

file_path = "./dst-3.0_16_1_hh_database.csv"

# чтение данных — в файле используется разделитель ";"
df = pd.read_csv(file_path, sep=";")

2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [ ]:
display(df.head())

3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

4. Обратите внимание на информацию о числе непустых значений.

In [ ]:
df.info()

5. Выведите основную статистическую информацию о столбцах.

In [ ]:
df.describe(include='all')

# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [ ]:
def extract_education_level(text):
    if pd.isnull(text):
        return None
    level = " ".join(text.split()[:2]).lower()
    return level

print("Смотрим уникальные записи нового столбца \"Образование\":")
df["Образование"] = df["Образование и ВУЗ"].apply(extract_education_level)
display(df["Образование"].unique())

print("Нормализуем категории:")
df["Образование"] = df["Образование"].replace({
    "высшее образование": "высшее",
    "среднее образование": "среднее"
})
display(df["Образование"].unique())

count_middle = df[df["Образование"] == "среднее"].shape[0]
print(f"Соискателей, имеющих среднее образование: {count_middle}\n")

print("Удаленяем столбцец \"Образование и ВУЗ\". Проверяем столбцы:")
df.drop(columns=["Образование и ВУЗ"], inplace=True)
display(df.columns)

2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000

Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*
`

In [ ]:
def extract_gender(text):
    if pd.isnull(text):
        return None

    # пол всегда стоит до первой запятой
    gender_raw = text.split(",")[0].strip().lower()

    # нормализуем
    if "муж" in gender_raw:
        return "М"
    elif "жен" in gender_raw:
        return "Ж"
    else:
        return None

print("Создаём столбец \"Пол\":")
df["Пол"] = df["Пол, возраст"].apply(extract_gender)
display(df["Пол"].value_counts())


print("Создаём столбец \"Возраст\" и проверяем 2 случайных строки:")
def extract_age(text):
    if pd.isnull(text):
        return None

    # возраст — вторая часть строки
    age_part = text.split(",")[1]
    age_number = age_part.strip().split()[0]

    return int(age_number)

df["Возраст"] = df["Пол, возраст"].apply(extract_age)
display(df["Возраст"].sample(2))


female_percent = (df[df["Пол"] == "Ж"].shape[0] / df.shape[0]) * 100
print(f"Процент женских резюме: {female_percent:.2f}%\n")

avg_age = df["Возраст"].mean()
print(f"Средний возраст соискателей: {avg_age:.1f}\n")


print("Удаление столбец \"Пол, возраст\". Проверяем столбцы:")
df.drop(columns=["Пол, возраст"], inplace=True)
display(df.columns)

3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
import re

pattern_years_months = re.compile(
    r"опыт работы\s+(\d+)\s*(?:год|года|лет)\s+(\d+)\s*(?:месяц|месяца|месяцев)",
    re.IGNORECASE
)

pattern_years_only = re.compile(
    r"опыт работы\s+(\d+)\s*(?:год|года|лет)",
    re.IGNORECASE
)

pattern_months_only = re.compile(
    r"опыт работы\s+(\d+)\s*(?:месяц|месяца|месяцев)",
    re.IGNORECASE
)

def extract_experience_months(text):

    # Обрабатываем пропуски
    if pd.isnull(text):
        return np.nan
    text = text.strip().lower()
    if text == "не указано":
        return np.nan
    
     # Пример "Опыт работы 3 года 11 месяцев"
    m = pattern_years_months.search(text)
    if m:
        years = int(m.group(1))
        months = int(m.group(2))
        return years * 12 + months
    
    # Пример "Опыт работы 4 года"
    m = pattern_years_only.search(text)
    if m:
        years = int(m.group(1))
        return years * 12
    
    # Пример "Опыт работы 11 месяцев"
    m = pattern_months_only.search(text)
    if m:
        months = int(m.group(1))
        return months
    
    # если совсем ничего не нашли
    return np.nan

print("Создаём новый столбец \"Опыт работы\" и смотрим 2 случайные строки:")
df["Опыт работы (месяц)"] = df["Опыт работы"].apply(extract_experience_months)
display(df["Опыт работы (месяц)"].sample(2))

median_experience = df["Опыт работы (месяц)"].median()
print(f"\nМедианный опыт работы (в месяцах): {median_experience:.0f}\n")

print("Удаляем исходный столбец \"Опыт работы\":")
df.drop(columns=["Опыт работы"], inplace=True)
display(df.columns)

4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [ ]:
# список городов-миллионников
million_cities = [
    'Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск',
    'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск',
    'Пермь', 'Воронеж', 'Волгоград'
]


def parse_city_relocation_travel(text):
    if pd.isnull(text):
        # город - Nan, готовность - False/False
        return np.nan, False, False

    parts = [p.strip() for p in text.split(",")]
    parts_low = [p.lower() for p in parts]

    # первое значение — город или город + метро
    city_raw = parts[0]
    # убираем метро
    city_clean = city_raw.split("м.")[0].strip()

    # категоризация
    if city_clean == "Москва":
        city = "Москва"
    elif city_clean == "Санкт-Петербург":
        city = "Санкт-Петербург"
    elif city_clean in million_cities:
        city = "город-миллионник"
    else:
        city = "другие"

    # готовность к переезду
    relocation = False
    for p in parts_low:
        if ("готов" in p or "хочу" in p) and "переезд" in p:
            relocation = True
        if "не готов" in p and "переезд" in p:
            relocation = False

    # готовность к переезду
    relocation = False

    for p in parts:
        p_low = p.lower()

        # интересуют "переезд" ИЛИ "переехать"
        if "переезд" in p_low or "переехать" in p_low:
            # отрицание имеет приоритет
            if "не готов" in p_low:
                relocation = False
            elif "готов" in p_low or "хочу" in p_low:
                relocation = True

    # готовность к командировкам
    travel = False
    travel_info_found = False

    for p in parts:
        p_low = p.lower()
        if "командиров" in p_low:
            travel_info_found = True
            if "не готов" in p_low:
                travel = False
            elif "готов" in p_low:
                travel = True

    # если отсутствует то False
    if not travel_info_found:
        travel = False

    return city, relocation, travel


print("Создаём новые столбцы \"Город\", \"Готовность к переезду\", \"Готовность к командировкам\" и смотрим 2 случайные строки в каждой:")
df["Город"], df["Готовность к переезду"], df["Готовность к командировкам"] = zip(
    *df["Город, переезд, командировки"].apply(parse_city_relocation_travel)
)
display(df[["Город", "Готовность к переезду", "Готовность к командировкам"]].sample(2))

spb_percent = (df[df["Город"] == "Санкт-Петербург"].shape[0] / df.shape[0]) * 100
print(f"Процент соискателей из Санкт-Петербурга: {spb_percent:.0f}%\n")

both_percent = (df[(df["Готовность к переезду"] == True) & (df["Готовность к командировкам"] == True)].shape[0] / df.shape[0]) * 100
print(f"Процент готовых и к переезду, и к командировкам: {both_percent:.0f}%")

print("Удаляем исходный столбец \"Город, переезд, командировки\":")
df.drop(columns=["Город, переезд, командировки"], inplace=True)
display(df.columns)

5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [ ]:
employment_categories = [
    "полная занятость",
    "частичная занятость",
    "проектная работа",
    "волонтерство",
    "стажировка"
]

schedule_categories = [
    "полный день",
    "сменный график",
    "гибкий график",
    "удалённая работа",
    "вахтовый метод"
]


for cat in employment_categories:
    df[cat] = df["Занятость"].str.lower().str.contains(cat)


for cat in schedule_categories:
    df[cat] = df["График"].str.lower().str.contains(cat)


q1 = df[(df["проектная работа"]) & (df["волонтерство"])].shape[0]
print(f"\n1) Людей, которые ищут проектную работу и волонтерство: {q1}")

q2 = df[(df["вахтовый метод"]) & (df["гибкий график"])].shape[0]
print(f"2) Людей, которые хотят работать вахтовым методом и с гибким графиком: {q2}")

print("Удаляем исходный столбцы \"Занятость\" и \"График\" и проверяем существующие столбцы:")
df.drop(columns=["Занятость", "График"], inplace=True)
display(df.columns)

6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
print("Загружаем файл ExchangeRates.csv и выводим столбцы:")
ex = pd.read_csv("ExchangeRates.csv")
display(ex.columns)

# приводим даты к datetime и оставляем только дату резюме
## в резюме
df["Обновление резюме"] = pd.to_datetime(df["Обновление резюме"], dayfirst=True)
df["resume_date"] = df["Обновление резюме"].dt.date
## в таблице курсов
ex["date"] = pd.to_datetime(ex["date"], format="%d/%m/%y").dt.date

# парсер ЗП
def parse_salary(x):
    if pd.isnull(x):
        return (np.nan, np.nan)

    x = x.replace(" ", "")

    num = ""
    for c in x:
        if c.isdigit():
            num += c
        else:
            break

    if num == "":
        return (np.nan, np.nan)

    amount = int(num)

    currency = x[len(num):].strip().lower()
    return amount, currency

print("Создаём столбцы \"salary_amount\" и \"salary_currency_raw\" и смотрим 2 случайные строки в каждой:")
df["salary_amount"], df["salary_currency_raw"] = zip(*df["ЗП"].apply(parse_salary))
display(df[["ЗП", "salary_amount", "salary_currency_raw"]].sample(2))

currency_map = {
    "руб.": "RUB",
    "бел.руб.": "BYN",
    "kzt": "KZT",
    "eur": "EUR",
    "usd": "USD",
    "грн.": "UAH",
    "сум": "UZS",
    "kgs": "KGS",
    "azn": "AZN"
}

df["salary_currency"] = df["salary_currency_raw"].map(currency_map)
print("Уникальные валюты после маппинга в ISO формат:")
display(df["salary_currency"].unique())

# объединяем резюме и курсы валют по валюте (ISO), дате резюме
df = df.merge(
    ex[["currency", "date", "proportion", "close"]],
    left_on=["salary_currency", "resume_date"],
    right_on=["currency", "date"],
    how="left"
)

# устанавливаем курс рубля
df.loc[df["salary_currency"] == "RUB", "close"] = 1
df.loc[df["salary_currency"] == "RUB", "proportion"] = 1

# проверяем пропуски после объединения
df[["salary_currency", "resume_date", "close", "proportion"]].isnull().sum()

print("Создаем столбец \"ЗП (руб)\"")
df["ЗП (руб)"] = df["salary_amount"] * df["close"] / df["proportion"]

median_salary = df["ЗП (руб)"].median() / 1000
print(f"Медианная зарплата (тыс. руб): {median_salary:.0f}")

# Удаляем всё лишнее
df.drop(columns=[
    "close",
    "currency",
    "date",
    "proportion",
    "resume_date",
    "salary_amount",
    "salary_currency",
    "salary_currency_raw",
    "ЗП"
], inplace=True)

print("Проверяем итоговую структуру данных")
df.info()

# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.hist(df["Возраст"], bins=30, edgecolor="black")
plt.title("Распределение возраста соискателей")
plt.xlabel("Возраст (лет)")
plt.ylabel("Количество соискателей")

plt.subplot(1, 2, 2)
sns.boxplot(x=df["Возраст"])
plt.title("Коробчатая диаграмма возраста соискателей")
plt.xlabel("Возраст (лет)")

plt.tight_layout()
plt.show()


print("Статистика признака 'Возраст':")

mode_age = df["Возраст"].mode()[0]
print(f" - Мода распределения: {mode_age} лет")

min_age = df["Возраст"].min()
max_age = df["Возраст"].max()
print(f" - Минимальное значение: {min_age}")
print(f" - Максимальное значение: {max_age}")

q1 = df["Возраст"].quantile(0.25)
q3 = df["Возраст"].quantile(0.75)
print(f" - Возраст большинства (IQR): {q1:.0f}–{q3:.0f} лет")

mode_age = df["Возраст"].mode()[0]
print(f" - Модальное значение возраста: {mode_age} лет")

anom = df[(df["Возраст"] < 14) | (df["Возраст"] > 70)]
print(f" - Потенциальные аномалии (<14 или >70): {anom.shape[0]}")
print(" - Уникальные аномальные значения возраста:", anom["Возраст"].unique())


Возраст соискателей чаще всего равен 30 годам. Основные значения лежат примерно в диапазоне 27–36 лет. Минимальный возраст — 14, максимальный — 100. Аномальными можно считать значения старше 70 лет.

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.hist(df["Опыт работы (месяц)"], bins=40, edgecolor="black")
plt.title("Распределение опыта работы (в месяцах)")
plt.xlabel("Опыт работы, месяцев")
plt.ylabel("Количество соискателей")

plt.subplot(1, 2, 2)
sns.boxplot(x=df["Опыт работы (месяц)"])
plt.title("Коробчатая диаграмма опыта работы")
plt.xlabel("Опыт работы, месяцев")

plt.tight_layout()
plt.show()


print("Статистика признака 'Опыт работы (месяц)':")

mode_exp = df["Опыт работы (месяц)"].mode()[0]
print(f" - Мода распределения: {mode_exp} месяцев")

min_exp = df["Опыт работы (месяц)"].min()
max_exp = df["Опыт работы (месяц)"].max()
print(f" - Минимальное значение: {min_exp}")
print(f" - Максимальное значение: {max_exp}")

q1 = df["Опыт работы (месяц)"].quantile(0.25)
q3 = df["Опыт работы (месяц)"].quantile(0.75)
print(f" - Опыт большинства соискателей: {q1:.0f}–{q3:.0f} месяцев")

anom = df[(df["Опыт работы (месяц)"] < 1) | (df["Опыт работы (месяц)"] > 500)]
print(f" - Количество аномалий: {anom.shape[0]}")
print(" - Уникальные 'аномальные' значения:")
print(anom["Опыт работы (месяц)"].unique())

Опыт работы чаще всего равен 81 месяцу. Основная масса соискателей имеет опыт примерно от 57 до 154 месяцев. Минимальное значение — 1 месяц, максимальное — 1188 месяцев, но такие большие значения выглядят как аномалии. К аномальным можно отнести значения опыта больше 500 месяцев, так как это эквивалентно более чем 40 годам стажа и встречается крайне редко.

3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака возраста? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.hist(df["ЗП (руб)"], bins=40, edgecolor="black")
plt.title("Распределение желаемой заработной платы (в рублях)")
plt.xlabel("Заработная плата, рубли")
plt.ylabel("Количество соискателей")

plt.subplot(1, 2, 2)
sns.boxplot(x=df["ЗП (руб)"])
plt.title("Коробчатая диаграмма заработной платы")
plt.xlabel("Заработная плата, рубли")

plt.tight_layout()
plt.show()


print("Статистика признака 'ЗП (руб)':")

min_salary = df["ЗП (руб)"].min()
max_salary = df["ЗП (руб)"].max()
print(f" - Минимальная зарплата: {min_salary:.0f} руб.")
print(f" - Максимальная зарплата: {max_salary:.0f} руб.")

q1 = df["ЗП (руб)"].quantile(0.25)
q3 = df["ЗП (руб)"].quantile(0.75)
print(f" - ЗП большинства соискателей (IQR): {q1/1000:.0f}–{q3/1000:.0f} тыс. руб.")

anom = df[df["ЗП (руб)"] > 1_000_000]
print(f" - Количество аномальных ЗП (>1 млн руб): {anom.shape[0]}")
print(" - Примеры аномальных значений:")
print(anom["ЗП (руб)"].unique()[:10])

Минимальная желаемая зарплата составляет 1 рубль, максимальная — около 24,3 млн рублей, что очевидно является аномалией. Основная часть соискателей указывает зарплату в диапазоне примерно от 37 до 95 тысяч рублей. Аномальными можно считать значения выше 1 млн рублей, их совсем немного.

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
df_filtered = df[df["ЗП (руб)"] < 1_000_000]

# Группируем и считаем медиану ЗП по образованию
median_salary_by_edu = df_filtered.groupby("Образование")["ЗП (руб)"].median().sort_values()

print("Медианные зарплаты по образованию (в рублях):")
display(median_salary_by_edu)

# Перевод в тысячи для удобства на графике
median_salary_by_edu_thousands = median_salary_by_edu / 1000

# Построение графика
plt.figure(figsize=(10, 5))
median_salary_by_edu_thousands.plot(kind="bar", edgecolor="black")

plt.title("Медианная желаемая заработная плата по уровням образования")
plt.ylabel("Зарплата, тыс. руб.")
plt.xlabel("Уровень образования")

plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

Самая низкая медианная зарплата — у соискателей со средним и средним специальным образованием (примерно 40 тысяч рублей).
Чуть выше зарплатные ожидания у людей с неоконченным высшим (около 50 тысяч).
Наиболее высокие ожидания у соискателей с высшим образованием — около 60 тысяч рублей.

В целом разница заметна, поэтому уровень образования действительно влияет на желаемую зарплату и может быть полезным признаком в моделях.

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
df_filtered = df[df["ЗП (руб)"] < 1_000_000]

plt.figure(figsize=(12, 6))
sns.boxplot(data=df_filtered, x="Город", y="ЗП (руб)")

plt.title("Распределение желаемой заработной платы по городам")
plt.xlabel("Город")
plt.ylabel("ЗП (рубли)")

plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


print("Медианные зарплаты по городам (в рублях):")
display(df_filtered.groupby("Город")["ЗП (руб)"].median().sort_values())

print("\nIQR (размах межквартильного диапазона):")
display(df_filtered.groupby("Город")["ЗП (руб)"].quantile(0.75) -
        df_filtered.groupby("Город")["ЗП (руб)"].quantile(0.25))

df_filtered = df[df["ЗП (руб)"] < 1_000_000]
target = 924000
row = df_filtered.iloc[(df_filtered["ЗП (руб)"] - target).abs().argsort()[:1]]
display(row[["Город", "ЗП (руб)"]])

Самая высокая медианная зарплата у соискателей из Москвы — около 85 тысяч рублей. Далее идёт Санкт-Петербург с медианой примерно 60 тысяч. В городах-миллионниках и других городах медианная зарплата почти одинаковая — около 40 тысяч рублей.

Размах зарплат тоже различается: в Москве он самый большой (IQR ≈ 90 тыс.), что говорит о сильной неоднородности ожиданий. В Петербурге разброс средний, а в остальных городах зарплаты более сгруппированы.

Таким образом, признак «Город» действительно важен для прогнозирования зарплаты — различия в уровне и разбросе заметные.

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
pivot = df.groupby(["Готовность к переезду", "Готовность к командировкам"])["ЗП (руб)"].median().unstack()

print("Медианные зарплаты по комбинациям признаков:")
display(pivot)

# Строим многоуровневую столбчатую диаграмму
plt.figure(figsize=(10, 6))
pivot.plot(kind="bar", figsize=(10, 6), edgecolor="black")

plt.title("Медианная заработная плата по готовности к переезду и командировкам")
plt.xlabel("Готовность к переезду")
plt.ylabel("Заработная плата, рубли")

plt.xticks(rotation=0)
plt.legend(title="Готовность к командировкам")
plt.tight_layout()
plt.show()

Самая низкая медианная зарплата у тех, кто не готов ни к переезду, ни к командировкам — около 40 тысяч рублей.
Если человек готов только к командировкам, зарплатные ожидания заметно выше — примерно 60 тысяч.
Готовность к переезду также повышает желаемую зарплату: среди готовых к переезду медиана растёт до 50 тысяч (без командировок) и до ≈66 тысяч, если человек готов и к переезду, и к командировкам.

В целом видно, что готовность к мобильности (переезды, командировки) связана с более высокими ожиданиями по зарплате, поэтому оба признака могут быть полезны для прогнозирования.

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
pivot = df.pivot_table(
    index="Возраст",
    columns="Образование",
    values="ЗП (руб)",
    aggfunc="median"
)

print("Сводная таблица медианной ЗП по возрасту и образованию:")
display(pivot)

plt.figure(figsize=(10, 8))
sns.heatmap(pivot, cmap="YlOrRd", linewidths=0.5)

plt.title("Тепловая карта медианной желаемой заработной платы\nпо возрасту и уровню образования")
plt.xlabel("Образование")
plt.ylabel("Возраст")

plt.tight_layout()
plt.show()

Самые высокие медианные зарплаты наблюдаются у соискателей с высшим образованием, особенно в возрасте примерно от 30 до 50 лет, где значения формируют самые красные области.
Для неоконченного высшего зарплаты тоже относительно высокие, но заметно ниже, чем у людей с высшим.
Уровни среднего и среднего специального образования практически во всех возрастах дают существенно более низкие ожидания.

Также видно, что с возрастом медианные зарплаты растут примерно до 40–45 лет, а затем постепенно снижаются.

В целом и возраст, и образование сильно влияют на желаемый уровень зарплаты, причём образование даёт наиболее устойчивую разницу между группами.

8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
df["Опыт (годы)"] = df["Опыт работы (месяц)"] / 12

plt.figure(figsize=(10, 6))

# Диаграмма рассеяния
plt.scatter(df["Возраст"], df["Опыт (годы)"], alpha=0.3, s=10)

# Добавляем линию возраст == опыт работы
plt.plot([0, 100], [0, 100], color="red", linestyle="--", linewidth=2)

plt.title("Зависимость опыта работы от возраста")
plt.xlabel("Возраст (лет)")
plt.ylabel("Опыт работы (лет)")

plt.xlim(0, 100)
plt.ylim(0, 100)

plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


anomalies = df[df["Опыт (годы)"] > df["Возраст"]]
print("Количество точек выше прямой:", anomalies.shape[0])

Большинство точек лежит существенно ниже линии «возраст = опыт», что логично.
Однако есть точки, находящиеся на линии и даже выше неё — это аномалии, потому что опыт работы не может быть равен или превышать возраст человека.

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
plt.figure(figsize=(16, 6))

# ГРАФИК 1 — ЗП vs Опыт
plt.subplot(1, 2, 1)

sns.scatterplot(
    data=df[df["ЗП (руб)"] < 1_000_000],
    x="Опыт (годы)",
    y="ЗП (руб)",
    hue="Образование",
    alpha=0.35
)

plt.title("Зависимость зарплаты от опыта работы и уровня образования")
plt.xlabel("Опыт работы (лет)")
plt.ylabel("Желаемая зарплата (руб)")
plt.grid(alpha=0.2)
plt.legend(title="Образование", fontsize=8)


# ГРАФИК 2 — ЗП по городам × переезд
plt.subplot(1, 2, 2)

sns.boxplot(
    data=df[df["ЗП (руб)"] < 1_000_000],
    x="Город",
    y="ЗП (руб)",
    hue="Готовность к переезду"
)

plt.title("Зависимость зарплаты от города и готовности к переезду")
plt.xlabel("Город")
plt.ylabel("Желаемая зарплата (руб)")
plt.grid(alpha=0.2)
plt.legend(title="Готов к переезду", fontsize=8)

plt.tight_layout()
plt.show()

Вывод 1 — опыт работы и образование
- Зарплатные ожидания растут вместе с опытом работы.
- Соискатели с высшим образованием на всех уровнях опыта хотят больше, чем остальные.
- Разрыв между уровнями образования особенно заметен после 10–15 лет опыта.

Вывод 2 — город и готовность к переезду
- В каждом типе города готовые к переезду хотят выше, чем те, кто не готов.
- Максимальные ожидания — у москвичей, готовых к переезду.
- В городах-миллионниках разница меньше, но тренд тот же: мобильность → выше запросы.

# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
duplicates_count = df.duplicated().sum()
print("Количество полных дубликатов:", duplicates_count)

df = df.drop_duplicates()
print("Размер таблицы после удаления дубликатов:", df.shape)

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

In [ ]:
print("Число пропусков в каждом столбце:\n")
display(df.isnull().sum())

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
df = df.dropna(subset=[
    "Последнее/нынешнее место работы",
    "Последняя/нынешняя должность"
])

median_exp = df["Опыт работы (месяц)"].median()
df["Опыт работы (месяц)"] = df["Опыт работы (месяц)"].fillna(median_exp)
print("Медиана опыта работы (месяц):", median_exp)

mean_exp = df["Опыт работы (месяц)"].mean()
print("Средний опыт работы (месяц):", round(mean_exp))

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
initial_size = df.shape[0]

df = df[(df["ЗП (руб)"] >= 1000) & (df["ЗП (руб)"] <= 1_000_000)]

print("Удалено строк:", initial_size - df.shape[0])
print("Размер таблицы после очистки:", df.shape)

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных

In [ ]:
df["Опыт (годы)"] = df["Опыт работы (месяц)"] / 12

initial_size = df.shape[0]

df = df[df["Опыт (годы)"] < df["Возраст"]]

print("Удалено строк:", initial_size - df.shape[0])
print("Размер таблицы после очистки:", df.shape)

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [ ]:
log_age = np.log(df["Возраст"])

plt.figure(figsize=(10, 5))
histplot = sns.histplot(log_age, bins=30, kde=True)

# среднее и стандартное отклонение
mu = log_age.mean()
sigma = log_age.std()

# линии 3 сигм
histplot.axvline(mu, color='k', lw=2, label="Среднее")
histplot.axvline(mu + 3*sigma, color='r', lw=2, linestyle="--", label="+3σ")
histplot.axvline(mu - 3*sigma, color='b', lw=2, linestyle="--", label="-3σ")

plt.title("Логарифмическое распределение возраста с линиями сигм")
plt.xlabel("log(Возраст)")
plt.ylabel("Количество")

plt.legend()
plt.show()

print("Среднее log-возраста:", mu)
print("Стандартное отклонение:", sigma)


left_bound = mu - 3*sigma
right_bound = mu + 4*sigma

print("Левая граница:", left_bound)
print("Правая граница:", right_bound)

# находим выбросы
outliers = df[(log_age < left_bound) | (log_age > right_bound)]

print("Число выбросов:", outliers.shape[0])
display(outliers[["Возраст"]].sort_values("Возраст"))

df = df[(log_age >= left_bound) & (log_age <= right_bound)]
print("Размер данных после удаления выбросов:", df.shape)


Логарифмическое распределение возраста асимметрично вправо — справа длинный хвост из более высоких возрастов.